In [1]:
#general python imports
import os
import dask
import matplotlib # Its not that this file is responsible for doing plotting, but it calls many modules that are, such that it needs to pre-empt
# setting of an appropriate backend.
matplotlib.use('Agg')
import sys
import numpy
import numpy as np
import scipy
import scipy.io as sio
import math
import re
#import requests
import time
from tabulate import tabulate
from textblob import TextBlob
import glob

#text analysis imports
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from nltk.tag.perceptron import PerceptronTagger
tagger = PerceptronTagger(load=False)

from nltk import word_tokenize,sent_tokenize
from nltk import sent_tokenize, word_tokenize, pos_tag
from nltk.probability import FreqDist
from nltk.classify import NaiveBayesClassifier
from nltk.corpus import subjectivity
from nltk.corpus import cmudict
from nltk.sentiment import SentimentAnalyzer
from nltk import compat
#from nltk.compat import Counter
#from nltk.draw import dispersion_plot

from bs4 import BeautifulSoup
import json

from textstat.textstat import textstat

import os
os.system('pip install natsort')
from natsort import natsorted, ns

########################################################################
########################################################################
########################################################################
# Set user parameters based on type of analysis
searchList = ['GMO','Genetically_Modified_Organism','Transgenic','Vaccine']
nweb = 4 #number of search websites being implemented (google, google scholar, bing, yahoo)
numURLs = 26 #number of URLs per search website  (number determined by 1.scrape code)

#also save these parameters for analysis purposes
spec_dict = { 'searchList':searchList, 'nweb':nweb, 'numURLs':numURLs}
handle = 'Data/analysisSpecs.mat'
scipy.io.savemat(handle, mdict=spec_dict, oned_as='row')
handle = None
TEXT_FOUNTAIN = False
########################################################################
########################################################################
########################################################################
#what are we analyzing???? - this is the list of text analysis features
infoDat = {}
infoDat[1,1] = "Number of Words"
infoDat[2,1] = "Number of Sentences"
infoDat[3,1] = "Frequency of Search Term"
infoDat[4,1] = "Sentiment Analysis"
infoDat[5,1] = "Subjectivity Analysis"
infoDat[6,1] = "Grade level"
infoDat[7,1] = "Flesch Reading Ease"
infoDat[8,1] = "SMOG Index"
infoDat[9,1] = "Coleman Liau"
infoDat[10,1] = "Automated Readability Index"
infoDat[11,1] = "Gunning Fog"
infoDat[12,1] = "Dale Chall Readability Score"
infoDat[13,1] = "Difficult Words"
infoDat[14,1] = "Linsear Write Formula"

#save these parameters for analysis purposes
infoDat = list(infoDat.items())
handle = 'Data/analysisInfo.mat'
scipy.io.savemat(handle, {'infoDat':infoDat})
handle = None
########################################################################
########################################################################
########################################################################
#set filePath below to specify where the data will be going after the code runs
fileLocation = os.getcwd()

if not os.path.exists(fileLocation):
    os.makedirs(fileLocation)


date_created = []
import pickle
import os

sl = [ (i, val) for i, val in enumerate(searchList) ]


def web_iter(sl):
    b,value = sl

    #s,value = sl
    print(s,value)
    nweb = 3
    obj_arr_add = []


    if b == 0:
        textName = "google_"
        print ("Google")
    elif b == 1:
        textName = "gScholar_"
        print ("Google Scholar")
    elif b == 2:
        textName = "bing_"
        print ("Bing")
    elif b == 3:
        textName = "yahoo_"
        print ("Yahoo")

    if not os.path.exists(str(fileLocation) + '/' + str(value) +'/'):
        os.makedirs(str(fileLocation) + '/' + str(value) +'/')
    os.chdir(fileLocation +str('/') + str(value) +'/')

    # grab all the file names ending with pickle suffix.
    list_of_files = natsorted(glob.glob(str(textName)+r'*.p'))
    # select only a subset of them.
    list_of_files =  natsorted(list_of_files[0:numURLs-1])
    #print(list_of_files)
    for p,fileName in enumerate(list_of_files):

        print ("Analyzing Search Engine " + str(b+1) + " of " + str(nweb) + ": Link " + str(p)); print ("");
        import pickle

        fileHandle = open(fileName, 'rb');
        file_contents = pickle.load(fileHandle)

        if TEXT_FOUNTAIN == True:
            # Recover the initial text file data, corresponding to both PDFs and html web page content.
            # Used to generate data for people who lack python, reproducibility.
            f = open(str(fileName)+'.txt','w')
            f.write(str(file_contents.encode('ascii','ignore')))
            f.close()

        if len(file_contents) == 2:
            date_created.append((str(fileName), file_contents[0]))
            url_text = file_contents[1]

        else:

            url_text = file_contents
        fileHandle.close()

        #initialize dataArray Dictionary
        urlDat = {}

        ########################################################################
        #remove unreadable characters
        url_text = url_text.replace("-", " ") #remove characters that nltk can't read
        textNum = re.findall(r'\d', url_text) #locate numbers that nltk cannot see to analyze
        for x in range(0,len(textNum)) :
            url_text.find(textNum[x])

        URLtext = word_tokenize(url_text)
        URLtext = [w.lower() for w in URLtext] #make everything lower case

        urlDat[1,1] = textstat.lexicon_count(str(url_text))
        #sentences
        sents = sent_tokenize(url_text) #split all of text in to sentences
        sents = [w.lower() for w in sents] #lowercase all

        urlDat[2,1] = len(sents) #determine number of sentences

        ########################################################################
        ##frequency distribtuion of text
        fdist = FreqDist(w.lower() for w in URLtext if w.isalpha()) #frequency distribution of words only

        # Bug fix
        # cast dict to list
        fd_temp = list(fdist.items())

        urlDat[3,1] = fdist[searchList[s].lower()] #frequency of search term

        #frequency of all words
        fAll = {}
        for x in range(0,len(fd_temp)):
            fAll[x,0], fAll[x,1] = [y.strip('}()",{:') for y in (str(fd_temp[x])).split(',')]

        #frequency of the most used n number of words
        frexMost = fdist.most_common(15) #show N most common words
        fM = {}
        for x in range(0,len(frexMost)) :
            fM[x,0], fM[x,1] = [y.strip('}()",{:') for y in (str(frexMost[x])).split(',')]

        ########################################################################
        #Sentiment and Subjectivity analysis
        testimonial = TextBlob(url_text)
        testimonial.sentiment

        urlDat[4,1] = testimonial.sentiment.polarity
        urlDat[5,1] = testimonial.sentiment.subjectivity

        ########################################################################
        #determine syllable count for all words in each sentece
        sentSyl = {}
        WperS = {}
        # for n,sent in enumerate(sents):
        # future use
        for n in range(0,len(sents)):

            #setup sent variable to analyze each sentence individually
            sent = sents[n] #select sentence n in total text
            sent = word_tokenize(sent) #tokenize sentence n in to words
            sent = [w.lower() for w in sent if w.isalpha()] #remove any non-text

            WperS[n] = len(sent) #number of words per sentence

            #syllable analysis

            for x in range(0,len(sent)):
                word = sent[x]

                # Count the syllables in the word.
                syllables = textstat.syllable_count(str(word))
                sentSyl[n,x] = syllables
        infoDat_ = {}
        '''
        infoDat[1,1] = "Number of Words"
        infoDat[2,1] = "Number of Sentences"
        infoDat[3,1] = "Frequency of Search Term"
        infoDat[4,1] = "Sentiment Analysis"
        infoDat[5,1] = "Subjectivity Analysis"
        '''
        if len(URLtext) != 0:
            #assert type(url_text) is not type(None)
            infoDat_["Grade level"] = textstat.flesch_kincaid_grade(str(url_text))
            infoDat_["Flesch Reading Ease"] = textstat.flesch_reading_ease(str(url_text))
            infoDat_["SMOG Index"] =  textstat.smog_index(str(url_text))
            infoDat_["Coleman Liau"] = textstat.coleman_liau_index(str(url_text))
            infoDat_["Automated Readability Index"] = textstat.automated_readability_index(str(url_text))
            infoDat_["Gunning Fog"] = textstat.gunning_fog(str(url_text))

            infoDat_["Dale Chall Readability Score"] = textstat.dale_chall_readability_score(str(url_text))
            infoDat_["Difficult Words"] = textstat.difficult_words(str(url_text))
            infoDat_["Linsear Write Formula"] = textstat.linsear_write_formula(str(url_text))

            urlDat[6,1]  = textstat.flesch_kincaid_grade(str(url_text))
            urlDat[7,1] = textstat.flesch_reading_ease(str(url_text))
            urlDat[8,1]  = textstat.smog_index(str(url_text))
            urlDat[9,1]  = textstat.coleman_liau_index(str(url_text))
            urlDat[10,1]  = textstat.automated_readability_index(str(url_text))
            urlDat[11,1] = textstat.gunning_fog(str(url_text))

            urlDat[12,1]  = textstat.dale_chall_readability_score(str(url_text))
            urlDat[13,1]  = textstat.difficult_words(str(url_text))
            urlDat[14,1]  = textstat.linsear_write_formula(str(url_text))

            urlDat = list(urlDat.items())
            sentSyl = list(sentSyl.items())
            WperS = list(WperS.items())
            fAll = list(fAll.items())
            fM = list(fM.items())

            assert len(fAll) != 0
            assert len(fM) != 0
            assert len(sentSyl) != 0

            obj_arr = np.array([urlDat, WperS, sentSyl, fM, fAll], dtype=np.object)
            obj_arr_add.append(obj_arr)
            print('dimensions change of object array: ',np.shape(obj_arr),np.shape(urlDat))

            assert len(obj_arr[-1])!= 0
            assert len(obj_arr[-2])!= 0
            assert len(obj_arr[-3])!= 0
            assert type(obj_arr) is not type(None)
            #print(obj_arr)

            f = open('last_iterator.p', 'wb')
            fi = [s,value, obj_arr]
            pickle.dump(fi,f)
            fi = None
            # File path is equivalent to Term.mat

        obj_arr_add = [ i for i in obj_arr_add  if i is not type(None) ]
        obj_arr = obj_arr_add[0]
        for oaa in obj_arr_add[1:-1]:
            obj_arr = np.vstack( [obj_arr, oaa])
        return obj_arr
    else:
        return None



[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


/opt/conda/lib/python3.6/site-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [3]:
def finish_up(obj_arr_add):
    '''
    This method doesn't really work but it should just convert matrices.
    '''
    import scipy

    handle = str('../Data/') + str(searchList[s]) + '.mat'
    print(s,value,str(searchList[s]))
    scipy.io.savemat(handle, {'obj_arr':obj_arr})
    os.chdir('../Data/')

    handle = 'file_name_versus_date.mat'
    scipy.io.savemat(handle, {'infoDated':date_created})

    os.system('octave read_mat.m')
    os.system('octave read_matSoft.m')



0 GMO
Google
0 b
1 Genetically_Modified_Organism

Analyzing Search Engine 1 of 3: Link 0
0 b
Google
2 Transgenic
0 b
Google
Analyzing Search Engine 1 of 3: Link 0
Analyzing Search Engine 1 of 3: Link 0

3 Vaccine
0 b

Google
Analyzing Search Engine 1 of 3: Link 0

dimensions change of object array:  (5,) (14, 2)
Analyzing Search Engine 1 of 3: Link 1

dimensions change of object array:  (5,) (14, 2)
Analyzing Search Engine 1 of 3: Link 1

dimensions change of object array:  (5,) (14, 2)
Analyzing Search Engine 1 of 3: Link 1

dimensions change of object array:  (5,) (14, 2)
Analyzing Search Engine 1 of 3: Link 1

dimensions change of object array:  (5,) (14, 2)
Analyzing Search Engine 1 of 3: Link 2

dimensions change of object array:  (5,) (14, 2)
Analyzing Search Engine 1 of 3: Link 2

dimensions change of object array:  (5,) (14, 2)
Analyzing Search Engine 1 of 3: Link 2

dimensions change of object array:  (5,) (14, 2)
Analyzing Search Engine 1 of 3: Link 2

dimensions change of ob

Analyzing Search Engine 1 of 3: Link 24

dimensions change of object array:  (5,) (14, 2)
Analyzing Search Engine 1 of 3: Link 22

dimensions change of object array:  (5,) (14, 2)
Analyzing Search Engine 1 of 3: Link 23

dimensions change of object array:  (5,) (14, 2)
Analyzing Search Engine 1 of 3: Link 23

dimensions change of object array:  (5,) (14, 2)
Analyzing Search Engine 1 of 3: Link 24

dimensions change of object array:  (5,) (14, 2)
dimensions change of object array:  (5,) (14, 2)
dimensions change of object array:  (5,) (14, 2)
Analyzing Search Engine 1 of 3: Link 23

dimensions change of object array:  (5,) (14, 2)
Analyzing Search Engine 1 of 3: Link 24

dimensions change of object array:  (5,) (14, 2)
Analyzing Search Engine 1 of 3: Link 24

dimensions change of object array:  (5,) (14, 2)
dimensions change of object array:  (5,) (14, 2)


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


,0
0,"[[{'Grade level': 12.9, 'Flesch Reading Ease':..."
1,"[[{'Grade level': 14.1, 'Flesch Reading Ease':..."
2,"[[{'Grade level': 12.6, 'Flesch Reading Ease':..."
3,"[[{'Grade level': 15.2, 'Flesch Reading Ease':..."


In [22]:
import dask.bag as db
grid = {}
grid['b']=[0,1,2,3]
grid['value']=['GMO','Genetically_Modified_Organism','Transgenic','Vaccine']
from sklearn.grid_search import ParameterGrid
grid = list(ParameterGrid(grid))
grid = [(dicti['b'],dicti['value']) for dicti in grid ]        
grid = db.from_sequence(grid,npartitions = 8)
obj_arr_add = list(db.map(web_iter,grid).compute())

import pandas as pd
df = pd.DataFrame(data=obj_arr_add)
print(obj_arr_add)
df

[{'b': 0, 'value': 'GMO'}, {'b': 0, 'value': 'Genetically_Modified_Organism'}, {'b': 0, 'value': 'Transgenic'}, {'b': 0, 'value': 'Vaccine'}, {'b': 1, 'value': 'GMO'}, {'b': 1, 'value': 'Genetically_Modified_Organism'}, {'b': 1, 'value': 'Transgenic'}, {'b': 1, 'value': 'Vaccine'}, {'b': 2, 'value': 'GMO'}, {'b': 2, 'value': 'Genetically_Modified_Organism'}, {'b': 2, 'value': 'Transgenic'}, {'b': 2, 'value': 'Vaccine'}, {'b': 3, 'value': 'GMO'}, {'b': 3, 'value': 'Genetically_Modified_Organism'}, {'b': 3, 'value': 'Transgenic'}, {'b': 3, 'value': 'Vaccine'}]
0 GMO
0 b
Google
0 Transgenic
0 b
Google
Analyzing Search Engine 1 of 3: Link 0
Analyzing Search Engine 1 of 3: Link 0


1 GMO
0 b
Google
1 Transgenic
Analyzing Search Engine 1 of 3: Link 0
0 b
2 GMO
0 b
Google

2 Transgenic
Google
Analyzing Search Engine 1 of 3: Link 0
0 b

Analyzing Search Engine 1 of 3: Link 0
Google
3 GMO
Analyzing Search Engine 1 of 3: Link 0

3 Transgenic
0 b

0 b
Google
Google
Analyzing Search Engine 1 of 3:

Analyzing Search Engine 1 of 3: Link 10

dimensions change of object array:  (5,) (14, 2)
Analyzing Search Engine 1 of 3: Link 12

dimensions change of object array:  (5,) (14, 2)
Analyzing Search Engine 1 of 3: Link 10

dimensions change of object array:  (5,) (14, 2)
Analyzing Search Engine 1 of 3: Link 10

dimensions change of object array:  (5,) (14, 2)
Analyzing Search Engine 1 of 3: Link 11

dimensions change of object array:  (5,) (14, 2)
Analyzing Search Engine 1 of 3: Link 11

dimensions change of object array:  (5,) (14, 2)
Analyzing Search Engine 1 of 3: Link 11

dimensions change of object array:  (5,) (14, 2)
Analyzing Search Engine 1 of 3: Link 11

dimensions change of object array:  (5,) (14, 2)
Analyzing Search Engine 1 of 3: Link 11

dimensions change of object array:  (5,) (14, 2)
Analyzing Search Engine 1 of 3: Link 12

dimensions change of object array:  (5,) (14, 2)
Analyzing Search Engine 1 of 3: Link 12

dimensions change of object array:  (5,) (14, 2)
Analyzing 


dimensions change of object array:  (5,) (14, 2)
Analyzing Search Engine 1 of 3: Link 22

dimensions change of object array:  (5,) (14, 2)
Analyzing Search Engine 1 of 3: Link 22

dimensions change of object array:  (5,) (14, 2)
Analyzing Search Engine 1 of 3: Link 22

dimensions change of object array:  (5,) (14, 2)
Analyzing Search Engine 1 of 3: Link 22

dimensions change of object array:  (5,) (14, 2)
Analyzing Search Engine 1 of 3: Link 23

dimensions change of object array:  (5,) (14, 2)
Analyzing Search Engine 1 of 3: Link 23

dimensions change of object array:  (5,) (14, 2)
Analyzing Search Engine 1 of 3: Link 23

dimensions change of object array:  (5,) (14, 2)
Analyzing Search Engine 1 of 3: Link 23

dimensions change of object array:  (5,) (14, 2)
Analyzing Search Engine 1 of 3: Link 23

dimensions change of object array:  (5,) (14, 2)
Analyzing Search Engine 1 of 3: Link 23

dimensions change of object array:  (5,) (14, 2)
Analyzing Search Engine 1 of 3: Link 23

dimension

Analyzing Search Engine 1 of 3: Link 10
Analyzing Search Engine 1 of 3: Link 8


dimensions change of object array:  (5,) (14, 2)
Analyzing Search Engine 1 of 3: Link 7

dimensions change of object array:  (5,) (14, 2)
Analyzing Search Engine 1 of 3: Link 8

dimensions change of object array:  (5,) (14, 2)
Analyzing Search Engine 1 of 3: Link 6

dimensions change of object array:  (5,) (14, 2)
Analyzing Search Engine 1 of 3: Link 11

dimensions change of object array:  (5,) (14, 2)
Analyzing Search Engine 1 of 3: Link 9

dimensions change of object array:  (5,) (14, 2)
Analyzing Search Engine 1 of 3: Link 9

dimensions change of object array:  (5,) (14, 2)
Analyzing Search Engine 1 of 3: Link 8

dimensions change of object array:  (5,) (14, 2)
Analyzing Search Engine 1 of 3: Link 7

dimensions change of object array:  (5,) (14, 2)
Analyzing Search Engine 1 of 3: Link 10

dimensions change of object array:  (5,) (14, 2)
Analyzing Search Engine 1 of 3: Link 10

dimensions change of objec

Analyzing Search Engine 1 of 3: Link 19

dimensions change of object array:  (5,) (14, 2)
Analyzing Search Engine 1 of 3: Link 20

dimensions change of object array:  (5,) (14, 2)
Analyzing Search Engine 1 of 3: Link 20

dimensions change of object array:  (5,) (14, 2)
Analyzing Search Engine 1 of 3: Link 20

dimensions change of object array:  (5,) (14, 2)
Analyzing Search Engine 1 of 3: Link 21

dimensions change of object array:  (5,) (14, 2)
Analyzing Search Engine 1 of 3: Link 19

dimensions change of object array:  (5,) (14, 2)
Analyzing Search Engine 1 of 3: Link 22

dimensions change of object array:  (5,) (14, 2)
Analyzing Search Engine 1 of 3: Link 20

dimensions change of object array:  (5,) (14, 2)
Analyzing Search Engine 1 of 3: Link 21

dimensions change of object array:  (5,) (14, 2)
Analyzing Search Engine 1 of 3: Link 20

dimensions change of object array:  (5,) (14, 2)
Analyzing Search Engine 1 of 3: Link 21

dimensions change of object array:  (5,) (14, 2)
Analyzing 

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


,0
0,"[[{'Grade level': 12.9, 'Flesch Reading Ease':..."
1,"[[{'Grade level': 14.1, 'Flesch Reading Ease':..."
2,"[[{'Grade level': 12.6, 'Flesch Reading Ease':..."
3,"[[{'Grade level': 15.2, 'Flesch Reading Ease':..."
4,"[[{'Grade level': 12.9, 'Flesch Reading Ease':..."
5,"[[{'Grade level': 14.1, 'Flesch Reading Ease':..."
6,"[[{'Grade level': 12.6, 'Flesch Reading Ease':..."
7,"[[{'Grade level': 15.2, 'Flesch Reading Ease':..."
8,"[[{'Grade level': 12.9, 'Flesch Reading Ease':..."
9,"[[{'Grade level': 14.1, 'Flesch Reading Ease':..."


In [23]:
print(len(obj_arr_add))
pd.DataFrame(data = obj_arr_add[0]) #reading scales
obj_arr_add[3] #word frequencies

df = pd.DataFrame(data=obj_arr_add[0])

df

16


,0,1,2,3,4
0,"{'Grade level': 12.9, 'Flesch Reading Ease': 2...","[(0, 14), (1, 6), (2, 4), (3, 9), (4, 41), (5,...","[((0, 0), 0.9), ((0, 1), 1.8), ((0, 2), 1.8), ...","[((0, 0), 'the'), ((0, 1), 192), ((1, 0), 'gm...","[((0, 0), 'gmo'), ((0, 1), 161), ((1, 0), 'go..."
1,"{'Grade level': 10.2, 'Flesch Reading Ease': 5...","[(0, 14), (1, 9), (2, 4), (3, 9), (4, 67), (5,...","[((0, 0), 0.9), ((0, 1), 1.8), ((0, 2), 1.8), ...","[((0, 0), 'the'), ((0, 1), 244), ((1, 0), 'gm...","[((0, 0), 'gmo'), ((0, 1), 137), ((1, 0), 'go..."
2,"{'Grade level': 24.9, 'Flesch Reading Ease': -...","[(0, 14), (1, 9), (2, 4), (3, 9), (4, 233), (5...","[((0, 0), 0.9), ((0, 1), 1.8), ((0, 2), 1.8), ...","[((0, 0), 'https'), ((0, 1), 223), ((1, 0), '...","[((0, 0), 'gmo'), ((0, 1), 156), ((1, 0), 'go..."
3,"{'Grade level': 9.7, 'Flesch Reading Ease': 43...","[(0, 14), (1, 9), (2, 4), (3, 9), (4, 17), (5,...","[((0, 0), 0.9), ((0, 1), 1.8), ((0, 2), 1.8), ...","[((0, 0), 'gmo'), ((0, 1), 129), ((1, 0), 'th...","[((0, 0), 'gmo'), ((0, 1), 129), ((1, 0), 'go..."
4,"{'Grade level': 12.4, 'Flesch Reading Ease': 3...","[(0, 14), (1, 9), (2, 4), (3, 9), (4, 25), (5,...","[((0, 0), 0.9), ((0, 1), 1.8), ((0, 2), 1.8), ...","[((0, 0), 'the'), ((0, 1), 361), ((1, 0), 'of...","[((0, 0), 'gmo'), ((0, 1), 111), ((1, 0), 'go..."
5,"{'Grade level': 7.8, 'Flesch Reading Ease': 65...","[(0, 14), (1, 9), (2, 4), (3, 9), (4, 65), (5,...","[((0, 0), 0.9), ((0, 1), 1.8), ((0, 2), 1.8), ...","[((0, 0), 'gmo'), ((0, 1), 126), ((1, 0), 'fr...","[((0, 0), 'gmo'), ((0, 1), 126), ((1, 0), 'go..."
6,"{'Grade level': 15.5, 'Flesch Reading Ease': -...","[(0, 14), (1, 6), (2, 4), (3, 9), (4, 21), (5,...","[((0, 0), 0.9), ((0, 1), 1.8), ((0, 2), 1.8), ...","[((0, 0), 'to'), ((0, 1), 4), ((1, 0), 'gmo')...","[((0, 0), 'gmo'), ((0, 1), 3), ((1, 0), 'goog..."
7,"{'Grade level': 14.8, 'Flesch Reading Ease': 1...","[(0, 58), (1, 40), (2, 27)]","[((0, 0), 0.9), ((0, 1), 2.7), ((0, 2), 3.6), ...","[((0, 0), 'search'), ((0, 1), 7), ((1, 0), 'y...","[((0, 0), 'search'), ((0, 1), 7), ((1, 0), 's..."
8,"{'Grade level': 14.8, 'Flesch Reading Ease': 1...","[(0, 58), (1, 40), (2, 27)]","[((0, 0), 0.9), ((0, 1), 2.7), ((0, 2), 3.6), ...","[((0, 0), 'search'), ((0, 1), 7), ((1, 0), 'y...","[((0, 0), 'search'), ((0, 1), 7), ((1, 0), 's..."
9,"{'Grade level': 16.5, 'Flesch Reading Ease': 4...","[(0, 39), (1, 40), (2, 21), (3, 7)]","[((0, 0), 0.9), ((0, 1), 2.7), ((0, 2), 3.6), ...","[((0, 0), 'search'), ((0, 1), 6), ((1, 0), 'i...","[((0, 0), 'search'), ((0, 1), 6), ((1, 0), 's..."


In [24]:
print(len(obj_arr_add))
pd.DataFrame(data = obj_arr_add[0]) #reading scales
obj_arr_add[3] #word frequencies

df = pd.DataFrame(data=obj_arr_add[3])
df.hist()

16


ValueError: num must be 1 <= num <= 0, not 1

In [ ]:
df
i=0
obj_arr_add = [ i for i in obj_arr_add  if i is not type(None) ]
obj_arr = obj_arr_add[0]

for oaa in obj_arr_add[1:-1]:
    obj_arr = np.vstack( [obj_arr, oaa])


In [ ]:

df
len(obj_arr)
np.shape(obj_arr)


In [ ]:
df[0].hist()

In [ ]:
obj_arr

In [ ]:
df[3]

In [4]:
sl

[(0, 'GMO'),
 (1, 'Genetically_Modified_Organism'),
 (2, 'Transgenic'),
 (3, 'Vaccine')]